In [2]:
import pandas as pd 
import re 
import numpy as np 
import tabula
import os

In [98]:
pdfs = sorted([i for i in os.listdir() if i.endswith('.pdf')])

## Parsing Fruits Info

In [144]:
gender = ['Males:', 'Females:', 'Males and females:']

In [145]:
fruit = ['total fruit', 'citrus, melons, berries', 'other fruit', 'fruit juice']

In [146]:
fruits = list()
yrs = list()

for i in range(len(pdfs)): 
    dfs = tabula.read_pdf(pdfs[i], pages = 1)
    # parsing the first page of each PDF, which is the fruit intake of each year
    a = dfs[0]
    a.dropna(axis = 1, inplace = True)
    # drop the NA column 
    print(a.iloc[6,0], i)
    # the start of the data frame
    # all the information below this row are the ones we need 
    a.drop(a.index[range(0,7)],axis = 0, inplace = True)
    # drop all the rows before this row 
    fruits.append(a)
    # append the parsed data frame into fruits list 
    yrs.append(re.findall('\d+_\d+', pdfs[i]))
    # save the year in the order of the data frame 
    # this is for the year column for each data frame later on 

|————————— Mean (Standard Error) —————————| 0
|————————— Mean (Standard Error) —————————| 1
|————————— Mean (Standard Error) —————————| 2
|————————— Mean (Standard Error) —————————| 3
|————————— Mean (Standard Error) —————————| 4
|————————— Mean (Standard Error) —————————| 5
|————————— Mean (Standard Error) —————————| 6


In [147]:
for dfIndex, df in enumerate(fruits): 
# loop through each data frame in fruits list 
    df.reset_index(drop = True, inplace = True)
    # reset the index since we dropped multiple rows on top 
    partition = list(df.index[df['Unnamed: 0'].isin(gender)])
    # the rows whose cells are the genders 
    df['gender'] = np.nan
    # gender column is created
    for i in range(len(partition)-1):
    # for each gender
        df.iloc[partition[i]: partition[i+1],1] = gender[i]
        # fill in the respective gender to the resepctive rows 
        if i ==2: 
            df.iloc[partition[i]: df.shape[0],1] = gender[i]
            
    df.gender[df.gender.isna()] = gender[2]
    # the rest of the gender column is filled with 'Male and female'
    
    df.gender = df.gender.apply(lambda x: x.rstrip(':'))
    # strip out ':' at the end of all the genders 
    df.drop(df.index[partition], axis = 0, inplace = True)
    # drop the gender rows that are not needed anymore 
    
    df['age'] = df['Unnamed: 0'].apply(lambda x: x.split('... ')[0])
    # split the first column with '...'
    # the elements on the left of the '...' are the age groups
    df.age = df.age.apply(lambda x: x.rstrip('...'))
    # strip out '...' from each age group values
    df['values'] = df['Unnamed: 0'].apply(lambda x: x.split('... ')[1])
    # all the numeric values are on the right side of '... '
    # create a seperate column for these values 
    
    for index in range(len(fruitNames)):
    # loop through each fruit names 
        df[fruitNames[index]] = df['values'].apply(lambda x: x.split()[index*2+1])
        # creating column for each fruit category 
        # splitting the rows with white space 
        # each odd-numbered element is the values we need 
        # assign the intake value of each fruit accordingly 
        
    df.drop(df.columns[[0,3]], axis = 1, inplace = True)
    # drop the columns we do not need anymore 
    df['year'] = yrs[dfIndex]*df.shape[0]
    # fill in the year column 

In [148]:
fruitDF = pd.concat(fruits)
# concatenate all the annual fruit intake data frames 

In [149]:
fruitDF

,gender,age,total fruit,"citrus, melons, berries",other fruit,fruit juice,year
1,Males,2 -5,1.37,0.14,0.49,0.73,2005_06
2,Males,6 - 11,1.06,0.15,0.51,0.41,2005_06
3,Males,12 - 19,0.98,0.16,0.34,0.48,2005_06
4,Males,20 - 29,0.83,0.07,0.27,0.49,2005_06
5,Males,30 - 39,0.86,0.16,0.30,0.39,2005_06
...,...,...,...,...,...,...,...
24,Females,20 and over,0.82,0.22,0.43,0.17,2017_18
25,Females,2 and over,0.88,0.23,0.44,0.20,2017_18
27,Males and females,2 - 19,1.10,0.25,0.51,0.34,2017_18
28,Males and females,20 and over,0.88,0.24,0.44,0.20,2017_18


## Parsing Vegetable Info

In [151]:
veg =['total vegetables', 'total starchy', 'potatoes', 'other starchy', 'total red and orange',\
     'tomatoes', 'other red and orange','dark green','other']

In [157]:
vegetables = list()

for i in range(len(pdfs)): 
    dfs = tabula.read_pdf(pdfs[i], pages = 2)
    # parsing the first page of each PDF, which is the fruit intake of each year
    a = dfs[0]
    #a.dropna(axis = 1, inplace = True)
    # drop the NA column 
    uselessRows = a.index[a['Unnamed: 0']==gender[0]][0]
    # the start of the data frame
    # all the information below this row are the ones we need 
    a.drop(a.index[range(0,uselessRows)], axis = 0, inplace = True)
    # drop the useless rows 
    a['values'] = a.iloc[:,0]+' '+a.iloc[:,1]
    # concatenate two columns to ease the process of splitting values 
    a.drop(a.columns[[0,1]], axis = 1, inplace = True)
    # drop the columns after concatenation
    a.reset_index(drop = True, inplace = True)
    # reset the indexes after dropping rows 
    vegetables.append(a)

In [158]:
for dfIndex, df in enumerate(vegetables): 
# loop through each data frame in fruits list 
    partition = list(df.index[df['values'].isna()])
    # the rows whose cells are the genders 
    df['gender'] = np.nan
    # gender column is created
    for i in range(len(partition)-1):
    # for each gender
        df.iloc[partition[i]: partition[i+1],1] = gender[i]
        # fill in the respective gender to the resepctive rows 
        if i ==2: 
            df.iloc[partition[i]: df.shape[0],1] = gender[i]
            
    df.gender[df.gender.isna()] = gender[2]
    # the rest of the gender column is filled with 'Male and female'
    
    df.gender = df.gender.apply(lambda x: x.rstrip(':'))
    # strip out ':' at the end of all the genders 
    df.drop(df.index[partition], axis = 0, inplace = True)
    # drop the gender rows that are not needed anymore 
    
    df['age'] = df['values'].apply(lambda x: x.split('... ')[0])
    # split the first column with '...'
    # the elements on the left of the '...' are the age groups
    df.age = df.age.apply(lambda x: x.rstrip('...'))
    # strip out '...' from each age group values
    df['values'] = df['values'].apply(lambda x: x.split('... ')[1])
    # all the numeric values are on the right side of '... '
    # create a seperate column for these values 
    
    for index in range(len(veg)):
    # loop through each fruit names 
        df[veg[index]] = df['values'].apply(lambda x: x.split()[index*2])
        # creating column for each fruit category 
        # splitting the rows with white space 
        # each odd-numbered element is the values we need 
        # assign the intake value of each fruit accordingly 
        
    df.drop(df.columns[0], axis = 1, inplace = True)
    # drop the columns we do not need anymore 
    df['year'] = yrs[dfIndex]*df.shape[0]
    # fill in the year column 

In [160]:
vegDF = pd.concat(vegetables)

## Parsing Grain Info

In [170]:
grainNames = ['total grains', 'whole grains', 'refined grains']

In [177]:
grains = list()

for i in range(len(pdfs)): 
    dfs = tabula.read_pdf(pdfs[i], pages = 3)
    # parsing the first page of each PDF, which is the fruit intake of each year
    a = dfs[0]
    #a.dropna(axis = 1, inplace = True)
    # drop the NA column 
    uselessRows = a.index[a['Unnamed: 0']==gender[0]][0]
    # the start of the data frame
    # all the information below this row are the ones we need 
    a.drop(a.index[range(0,uselessRows)], axis = 0, inplace = True)
    # drop the useless rows 
    a.dropna(axis =1, inplace = True)
    
    a.reset_index(drop = True, inplace = True)
    # reset the indexes after dropping rows 
    grains.append(a)

In [178]:
for dfIndex, df in enumerate(grains): 
# loop through each data frame in grains list 
    partition = list(df.index[df['Unnamed: 0'].isin(gender)])
    # the rows whose cells are the genders 
    df['gender'] = np.nan
    # gender column is created
    for i in range(len(partition)-1):
    # for each gender
        df.iloc[partition[i]: partition[i+1],1] = gender[i]
        # fill in the respective gender to the resepctive rows 
        if i ==2: 
            df.iloc[partition[i]: df.shape[0],1] = gender[i]
            
    df.gender[df.gender.isna()] = gender[2]
    # the rest of the gender column is filled with 'Male and female'
    
    df.gender = df.gender.apply(lambda x: x.rstrip(':'))
    # strip out ':' at the end of all the genders 
    df.drop(df.index[partition], axis = 0, inplace = True)
    # drop the gender rows that are not needed anymore 
    
    df['age'] = df['Unnamed: 0'].apply(lambda x: x.split('... ')[0])
    # split the first column with '...'
    # the elements on the left of the '...' are the age groups
    df.age = df.age.apply(lambda x: x.rstrip('...'))
    # strip out '...' from each age group values
    df['values'] = df['Unnamed: 0'].apply(lambda x: x.split('... ')[1])
    # all the numeric values are on the right side of '... '
    # create a seperate column for these values 
    
    for index in range(len(grainNames)):
    # loop through each fruit names 
        df[grainNames[index]] = df['values'].apply(lambda x: x.split()[index*2])
        # creating column for each fruit category 
        # splitting the rows with white space 
        # each odd-numbered element is the values we need 
        # assign the intake value of each fruit accordingly 
        
    df.drop(df.columns[0], axis = 1, inplace = True)
    # drop the columns we do not need anymore 
    df['year'] = yrs[dfIndex]*df.shape[0]
    # fill in the year column 
    df.drop(['values'], axis = 1, inplace = True)

In [180]:
grainDF = pd.concat(grains)

In [181]:
grainDF

,gender,age,total grains,whole grains,refined grains,year
1,Males,2 -5,5.08,0.55,4.53,2005_06
2,Males,6 - 11,7.14,0.55,6.59,2005_06
3,Males,12 - 19,9.05,0.51,8.53,2005_06
4,Males,20 - 29,8.79,0.69,8.10,2005_06
5,Males,30 - 39,9.00,0.73,8.28,2005_06
...,...,...,...,...,...,...
24,Females,20 and over,5.58,0.73,4.85,2017_18
25,Females,2 and over,5.69,0.75,4.95,2017_18
27,Males and females,2 - 19,6.75,0.85,5.90,2017_18
28,Males and females,20 and over,6.60,0.84,5.76,2017_18


## Parsing Grain Info

In [182]:
dairyNames = ['total dairy', 'fluid milk', 'cheese', 'yoghurt']

In [189]:
dairies = list()

for i in range(len(pdfs)): 
    dfs = tabula.read_pdf(pdfs[i], pages = 4)
    # parsing the first page of each PDF, which is the fruit intake of each year
    a = dfs[0]
    #a.dropna(axis = 1, inplace = True)
    # drop the NA column 
    uselessRows = a.index[a['Unnamed: 0']==gender[0]][0]
    # the start of the data frame
    # all the information below this row are the ones we need 
    a.drop(a.index[range(0,uselessRows)], axis = 0, inplace = True)
    # drop the useless rows 
    a.dropna(axis =1, inplace = True)
    
    a.reset_index(drop = True, inplace = True)
    # reset the indexes after dropping rows 
    dairies.append(a)

In [191]:
for dfIndex, df in enumerate(dairies): 
# loop through each data frame in grains list 
    partition = list(df.index[df['Unnamed: 0'].isin(gender)])
    # the rows whose cells are the genders 
    df['gender'] = np.nan
    # gender column is created
    for i in range(len(partition)-1):
    # for each gender
        df.iloc[partition[i]: partition[i+1],1] = gender[i]
        # fill in the respective gender to the resepctive rows 
        if i ==2: 
            df.iloc[partition[i]: df.shape[0],1] = gender[i]
            
    df.gender[df.gender.isna()] = gender[2]
    # the rest of the gender column is filled with 'Male and female'
    
    df.gender = df.gender.apply(lambda x: x.rstrip(':'))
    # strip out ':' at the end of all the genders 
    df.drop(df.index[partition], axis = 0, inplace = True)
    # drop the gender rows that are not needed anymore 
    
    df['age'] = df['Unnamed: 0'].apply(lambda x: x.split('... ')[0])
    # split the first column with '...'
    # the elements on the left of the '...' are the age groups
    df.age = df.age.apply(lambda x: x.rstrip('...'))
    # strip out '...' from each age group values
    df['values'] = df['Unnamed: 0'].apply(lambda x: x.split('... ')[1])
    # all the numeric values are on the right side of '... '
    # create a seperate column for these values 
    
    for index in range(len(dairyNames)):
    # loop through each fruit names 
        df[dairyNames[index]] = df['values'].apply(lambda x: x.split()[index*2])
        # creating column for each fruit category 
        # splitting the rows with white space 
        # each odd-numbered element is the values we need 
        # assign the intake value of each fruit accordingly 
        
    df.drop(df.columns[0], axis = 1, inplace = True)
    # drop the columns we do not need anymore 
    df['year'] = yrs[dfIndex]*df.shape[0]
    # fill in the year column 
    df.drop(['values'], axis = 1, inplace = True)

In [194]:
dairyDF = pd.concat(dairies)

In [195]:
dairyDF

,gender,age,total dairy,fluid milk,cheese,yoghurt,year
1,Males,2 -5,2.34,1.75,0.51,0.06,2005_06
2,Males,6 - 11,2.40,1.59,0.74,0.05*,2005_06
3,Males,12 - 19,2.47,1.35,1.06,0.04*,2005_06
4,Males,20 - 29,1.94,0.87,1.02,0.04,2005_06
5,Males,30 - 39,2.19,1.14,0.97,0.06*,2005_06
...,...,...,...,...,...,...,...
24,Females,20 and over,1.27,0.52,0.65,0.07,2017_18
25,Females,2 and over,1.34,0.60,0.64,0.07,2017_18
27,Males and females,2 - 19,1.81,1.01,0.71,0.06,2017_18
28,Males and females,20 and over,1.44,0.59,0.75,0.06,2017_18


## Parsing Protein Food

In [196]:
protein1 = ['total protein foods', 'Total Meat, Poultry, and Seafood', 'meat', 'poultry', 'cured meat',\
           'seafood low n-3', 'seafood high n-3', 'organ meat']

In [200]:
protein1DFs = list()

for i in range(len(pdfs)): 
    dfs = tabula.read_pdf(pdfs[i], pages = 5)
    # parsing the first page of each PDF, which is the fruit intake of each year
    a = dfs[0]
    #a.dropna(axis = 1, inplace = True)
    # drop the NA column 
    uselessRows = a.index[a['Unnamed: 0']==gender[0]][0]
    # the start of the data frame
    # all the information below this row are the ones we need 
    a.drop(a.index[range(0,uselessRows)], axis = 0, inplace = True)
    # drop the useless rows 
    a['values'] = a.iloc[:,0]+' '+a.iloc[:,1]
    # concatenate two columns to ease the process of splitting values 
    a.drop(a.columns[[0,1]], axis = 1, inplace = True)
    # drop the columns after concatenation
    a.reset_index(drop = True, inplace = True)
    # reset the indexes after dropping rows 
    protein1DFs.append(a)

In [202]:
for dfIndex, df in enumerate(protein1DFs): 
# loop through each data frame in fruits list 
    partition = list(df.index[df['values'].isna()])
    # the rows whose cells are the genders 
    df['gender'] = np.nan
    # gender column is created
    for i in range(len(partition)-1):
    # for each gender
        df.iloc[partition[i]: partition[i+1],1] = gender[i]
        # fill in the respective gender to the resepctive rows 
        if i ==2: 
            df.iloc[partition[i]: df.shape[0],1] = gender[i]
            
    df.gender[df.gender.isna()] = gender[2]
    # the rest of the gender column is filled with 'Male and female'
    
    df.gender = df.gender.apply(lambda x: x.rstrip(':'))
    # strip out ':' at the end of all the genders 
    df.drop(df.index[partition], axis = 0, inplace = True)
    # drop the gender rows that are not needed anymore 
    
    df['age'] = df['values'].apply(lambda x: x.split('... ')[0])
    # split the first column with '...'
    # the elements on the left of the '...' are the age groups
    df.age = df.age.apply(lambda x: x.rstrip('...'))
    # strip out '...' from each age group values
    df['values'] = df['values'].apply(lambda x: x.split('... ')[1])
    # all the numeric values are on the right side of '... '
    # create a seperate column for these values 
    
    for index in range(len(protein1)):
    # loop through each fruit names 
        df[protein1[index]] = df['values'].apply(lambda x: x.split()[index*2])
        # creating column for each fruit category 
        # splitting the rows with white space 
        # each odd-numbered element is the values we need 
        # assign the intake value of each fruit accordingly 
        
    df.drop(df.columns[0], axis = 1, inplace = True)
    # drop the columns we do not need anymore 
    df['year'] = yrs[dfIndex]*df.shape[0]
    # fill in the year column 

In [215]:
protein1DF = pd.concat(protein1DFs)

In [204]:
p2 = ['eggs', 'nuts and seeds', 'soybean products']

In [207]:
protein2DFs = list()

for i in range(len(pdfs)): 
    dfs = tabula.read_pdf(pdfs[i], pages = 6)
    # parsing the first page of each PDF, which is the fruit intake of each year
    a = dfs[0]
    #a.dropna(axis = 1, inplace = True)
    # drop the NA column 
    uselessRows = a.index[a['Unnamed: 0']==gender[0]][0]
    # the start of the data frame
    # all the information below this row are the ones we need 
    a.drop(a.index[range(0,uselessRows)], axis = 0, inplace = True)
    # drop the useless rows 
    a.dropna(axis =1, inplace = True)
    # drop empty column
    
    a.reset_index(drop = True, inplace = True)
    # reset the indexes after dropping rows 
    protein2DFs.append(a)

In [210]:
for dfIndex, df in enumerate(protein2DFs): 
# loop through each data frame in grains list 
    partition = list(df.index[df['Unnamed: 0'].isin(gender)])
    # the rows whose cells are the genders 
    df['gender'] = np.nan
    # gender column is created
    for i in range(len(partition)-1):
    # for each gender
        df.iloc[partition[i]: partition[i+1],1] = gender[i]
        # fill in the respective gender to the resepctive rows 
        if i ==2: 
            df.iloc[partition[i]: df.shape[0],1] = gender[i]
            
    df.gender[df.gender.isna()] = gender[2]
    # the rest of the gender column is filled with 'Male and female'
    
    df.gender = df.gender.apply(lambda x: x.rstrip(':'))
    # strip out ':' at the end of all the genders 
    df.drop(df.index[partition], axis = 0, inplace = True)
    # drop the gender rows that are not needed anymore 
    
    df['age'] = df['Unnamed: 0'].apply(lambda x: x.split('... ')[0])
    # split the first column with '...'
    # the elements on the left of the '...' are the age groups
    df.age = df.age.apply(lambda x: x.rstrip('...'))
    # strip out '...' from each age group values
    df['values'] = df['Unnamed: 0'].apply(lambda x: x.split('... ')[1])
    # all the numeric values are on the right side of '... '
    # create a seperate column for these values 
    
    for index in range(len(p2)):
    # loop through each fruit names 
        df[p2[index]] = df['values'].apply(lambda x: x.split()[index*2])
        # creating column for each fruit category 
        # splitting the rows with white space 
        # each odd-numbered element is the values we need 
        # assign the intake value of each fruit accordingly 
        
    df.drop(df.columns[0], axis = 1, inplace = True)
    # drop the columns we do not need anymore 
    df['year'] = yrs[dfIndex]*df.shape[0]
    # fill in the year column 
    df.drop(['values'], axis = 1, inplace = True)

In [212]:
protein2DFs = pd.concat(protein2DFs)

In [218]:
proteinDF = pd.merge(protein2DFs, protein1DF, on=["year", "age",'gender'])

## Parse Oil and Others

In [219]:
oil = ['oil', 'solid fats', 'added sugars', 'alcoholic drinks']

In [241]:
oilDfs = list()

for i in range(len(pdfs)): 
    dfs = tabula.read_pdf(pdfs[i], pages = 8)
    # parsing the first page of each PDF, which is the fruit intake of each year
    a = dfs[0]
    a.columns = ['values']
    # drop the NA column 
    uselessRows = a.index[a['values']==gender[0]][0]
    # the start of the data frame
    # all the information below this row are the ones we need 
    a.drop(a.index[range(0,uselessRows)], axis = 0, inplace = True)
    # drop the useless rows 
    a.dropna(axis =1, inplace = True)
    # drop empty column
    
    a.reset_index(drop = True, inplace = True)
    # reset the indexes after dropping rows 
    oilDfs.append(a)

In [242]:
for dfIndex, df in enumerate(oilDfs): 
# loop through each data frame in fruits list 
    partition = list(df.index[df['values'].isin(gender)])
    # the rows whose cells are the genders 
    df['gender'] = np.nan
    # gender column is created
    for i in range(len(partition)-1):
    # for each gender
        df.iloc[partition[i]: partition[i+1],1] = gender[i]
        # fill in the respective gender to the resepctive rows 
        if i ==2: 
            df.iloc[partition[i]: df.shape[0],1] = gender[i]
            
    df.gender[df.gender.isna()] = gender[2]
    # the rest of the gender column is filled with 'Male and female'
    
    df.gender = df.gender.apply(lambda x: x.rstrip(':'))
    # strip out ':' at the end of all the genders 
    df.drop(df.index[partition], axis = 0, inplace = True)
    # drop the gender rows that are not needed anymore 
    
    df['age'] = df['values'].apply(lambda x: x.split('... ')[0])
    # split the first column with '...'
    # the elements on the left of the '...' are the age groups
    df.age = df.age.apply(lambda x: x.rstrip('...'))
    # strip out '...' from each age group values
    df['values'] = df['values'].apply(lambda x: x.split('... ')[1])
    # all the numeric values are on the right side of '... '
    # create a seperate column for these values 
    
    for index in range(len(oil)):
    # loop through each fruit names 
        df[oil[index]] = df['values'].apply(lambda x: x.split()[index*2])
        # creating column for each fruit category 
        # splitting the rows with white space 
        # each odd-numbered element is the values we need 
        # assign the intake value of each fruit accordingly 
        
    df.drop(df.columns[0], axis = 1, inplace = True)
    # drop the columns we do not need anymore 
    df['year'] = yrs[dfIndex]*df.shape[0]
    # fill in the year column 

In [244]:
oilDF = pd.concat(oilDfs)

In [245]:
tb_1 = pd.merge(oilDF, proteinDF, on=["year", "age",'gender'])
tb_2 = pd.merge(dairyDF, grainDF, on=["year", "age",'gender'])
tb_3 = pd.merge(vegDF, fruitDF, on=["year", "age",'gender'])
tb_1_2 = pd.merge(tb_1, tb_2, on=["year", "age",'gender'])
finalTB = pd.merge(tb_1_2, tb_3, on=["year", "age",'gender'])

In [248]:
finalTB.head(5)

,gender,age,oil,solid fats,added sugars,alcoholic drinks,year,eggs,nuts and seeds,soybean products,...,other starchy,total red and orange,tomatoes,other red and orange,dark green,other,total fruit,"citrus, melons, berries",other fruit,fruit juice
0,Males,2 -5,14.16,31.45,14.72,0.00,2005_06,0.26,0.26,0.01,...,0.06,0.22,0.18,0.04,0.03,0.16,1.37,0.14,0.49,0.73
1,Males,6 - 11,20.63,42.55,21.61,0.00,2005_06,0.36,0.53*,0.01,...,0.05,0.33,0.25,0.08*,0.05,0.29,1.06,0.15,0.51,0.41
2,Males,12 - 19,23.26,55.31,31.09,0.20,2005_06,0.46,0.46,0.04*,...,0.06,0.43,0.38,0.06,0.05,0.33,0.98,0.16,0.34,0.48
3,Males,20 - 29,26.09,50.73,28.48,1.41,2005_06,0.47,0.51,0.08*,...,0.06,0.52,0.47,0.05*,0.10,0.64,0.83,0.07,0.27,0.49
4,Males,30 - 39,27.93,59.34,25.72,1.41,2005_06,0.62,0.73,0.10*,...,0.10,0.56,0.47,0.08,0.07,0.63,0.86,0.16,0.30,0.39


In [249]:
finalTB.to_csv('us_FoodG_gender.csv', index = False)